<img align="left" src="https://lever-client-logos.s3.amazonaws.com/864372b1-534c-480e-acd5-9711f850815c-1524247202159.png" width=200>
<br></br>

# Hyperparameter Tuning

## *Data Science Unit 4 Sprint 2 Assignment 4*

## Your Mission, should you choose to accept it...

To hyperparameter tune and extract every ounce of accuracy out of this telecom customer churn dataset: [Available Here](https://lambdaschool-data-science.s3.amazonaws.com/telco-churn/WA_Fn-UseC_-Telco-Customer-Churn+(1).csv)

## Requirements

- Load the data
- Clean the data if necessary (it will be)
- Create and fit a baseline Keras MLP model to the data.
- Hyperparameter tune (at least) the following parameters:
 - batch_size
 - training epochs
 - optimizer
 - learning rate (if applicable to optimizer)
 - momentum (if applicable to optimizer)
 - activation functions
 - network weight initialization
 - dropout regularization
 - number of neurons in the hidden layer
 
 You must use Grid Search and Cross Validation for your initial pass of the above hyperparameters
 
 Try and get the maximum accuracy possible out of this data! You'll save big telecoms millions! Doesn't that sound great?


In [1]:
##### Your Code Here #####

#Essential packages
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import category_encoders as ce
from sklearn.model_selection import GridSearchCV, train_test_split 
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import StandardScaler

#Tensorflow packages
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout, Flatten
from tensorflow.keras.wrappers.scikit_learn import KerasClassifier
from tensorflow.keras.callbacks import EarlyStopping

In [2]:
#Load data
df = pd.read_csv('WA_Fn-UseC_-Telco-Customer-Churn+.csv')
df.head()

,customerID,gender,SeniorCitizen,Partner,Dependents,tenure,PhoneService,MultipleLines,InternetService,OnlineSecurity,...,DeviceProtection,TechSupport,StreamingTV,StreamingMovies,Contract,PaperlessBilling,PaymentMethod,MonthlyCharges,TotalCharges,Churn
0,7590-VHVEG,Female,0,Yes,No,1,No,No phone service,DSL,No,...,No,No,No,No,Month-to-month,Yes,Electronic check,29.85,29.85,No
1,5575-GNVDE,Male,0,No,No,34,Yes,No,DSL,Yes,...,Yes,No,No,No,One year,No,Mailed check,56.95,1889.5,No
2,3668-QPYBK,Male,0,No,No,2,Yes,No,DSL,Yes,...,No,No,No,No,Month-to-month,Yes,Mailed check,53.85,108.15,Yes
3,7795-CFOCW,Male,0,No,No,45,No,No phone service,DSL,Yes,...,Yes,Yes,No,No,One year,No,Bank transfer (automatic),42.30,1840.75,No
4,9237-HQITU,Female,0,No,No,2,Yes,No,Fiber optic,No,...,No,No,No,No,Month-to-month,Yes,Electronic check,70.70,151.65,Yes


In [3]:
df.columns

Index(['customerID', 'gender', 'SeniorCitizen', 'Partner', 'Dependents',
       'tenure', 'PhoneService', 'MultipleLines', 'InternetService',
       'OnlineSecurity', 'OnlineBackup', 'DeviceProtection', 'TechSupport',
       'StreamingTV', 'StreamingMovies', 'Contract', 'PaperlessBilling',
       'PaymentMethod', 'MonthlyCharges', 'TotalCharges', 'Churn'],
      dtype='object')

In [4]:
print(df.isnull().sum())
print(df.shape)
print(df.dtypes)

customerID          0
gender              0
SeniorCitizen       0
Partner             0
Dependents          0
tenure              0
PhoneService        0
MultipleLines       0
InternetService     0
OnlineSecurity      0
OnlineBackup        0
DeviceProtection    0
TechSupport         0
StreamingTV         0
StreamingMovies     0
Contract            0
PaperlessBilling    0
PaymentMethod       0
MonthlyCharges      0
TotalCharges        0
Churn               0
dtype: int64
(7043, 21)
customerID           object
gender               object
SeniorCitizen         int64
Partner              object
Dependents           object
tenure                int64
PhoneService         object
MultipleLines        object
InternetService      object
OnlineSecurity       object
OnlineBackup         object
DeviceProtection     object
TechSupport          object
StreamingTV          object
StreamingMovies      object
Contract             object
PaperlessBilling     object
PaymentMethod        object
MonthlyCha

In [5]:
#Check values of all columns
col_list = ['MultipleLines', 'InternetService', 'Contract', 'PaymentMethod','gender', 'Partner', 'Dependents', 'PhoneService', 'PaperlessBilling', 'TechSupport', 'StreamingTV', 'StreamingMovies',
             'OnlineBackup', 'OnlineSecurity', 'DeviceProtection', 'Churn']
for b in col_list:
    print(df[b].value_counts())

No                  3390
Yes                 2971
No phone service     682
Name: MultipleLines, dtype: int64
Fiber optic    3096
DSL            2421
No             1526
Name: InternetService, dtype: int64
Month-to-month    3875
Two year          1695
One year          1473
Name: Contract, dtype: int64
Electronic check             2365
Mailed check                 1612
Bank transfer (automatic)    1544
Credit card (automatic)      1522
Name: PaymentMethod, dtype: int64
Male      3555
Female    3488
Name: gender, dtype: int64
No     3641
Yes    3402
Name: Partner, dtype: int64
No     4933
Yes    2110
Name: Dependents, dtype: int64
Yes    6361
No      682
Name: PhoneService, dtype: int64
Yes    4171
No     2872
Name: PaperlessBilling, dtype: int64
No                     3473
Yes                    2044
No internet service    1526
Name: TechSupport, dtype: int64
No                     2810
Yes                    2707
No internet service    1526
Name: StreamingTV, dtype: int64
No           

In [6]:
#Wrangle function to clean up data
def wrangle(X):
    
    #Dropping customer ID column
    X.drop(['customerID'], axis=1, inplace=True)
    
    #Factorizing all other columns
    col_1 = ['InternetService', 'Contract', 'PaymentMethod']
    for a in col_1:
        X[a] = pd.factorize(X[a])[0]
    
    col_2 = ['MultipleLines', 'gender', 'Partner', 'Dependents', 'PhoneService', 'PaperlessBilling', 'TechSupport',
            'StreamingTV', 'StreamingMovies', 'OnlineBackup', 'OnlineSecurity', 'DeviceProtection', 'Churn']
    for b in col_2:
        X[b] = X[b].replace({'Yes':1, 'Male':1, 'No':0, 'Female':0, 'No internet service':2, 'No phone service':2})
    
    #Change TotalCharges column type to float
    X['TotalCharges'] = pd.to_numeric(X['TotalCharges'], errors='coerce')
    
    return X

In [7]:
df = wrangle(df)
df.head()

,gender,SeniorCitizen,Partner,Dependents,tenure,PhoneService,MultipleLines,InternetService,OnlineSecurity,OnlineBackup,DeviceProtection,TechSupport,StreamingTV,StreamingMovies,Contract,PaperlessBilling,PaymentMethod,MonthlyCharges,TotalCharges,Churn
0,0,0,1,0,1,0,2,0,0,1,0,0,0,0,0,1,0,29.85,29.85,0
1,1,0,0,0,34,1,0,0,1,0,1,0,0,0,1,0,1,56.95,1889.50,0
2,1,0,0,0,2,1,0,0,1,1,0,0,0,0,0,1,1,53.85,108.15,1
3,1,0,0,0,45,0,2,0,1,0,1,1,0,0,1,0,2,42.30,1840.75,0
4,0,0,0,0,2,1,0,1,0,0,0,0,0,0,0,1,0,70.70,151.65,1


In [8]:
df.dtypes

gender                int64
SeniorCitizen         int64
Partner               int64
Dependents            int64
tenure                int64
PhoneService          int64
MultipleLines         int64
InternetService       int64
OnlineSecurity        int64
OnlineBackup          int64
DeviceProtection      int64
TechSupport           int64
StreamingTV           int64
StreamingMovies       int64
Contract              int64
PaperlessBilling      int64
PaymentMethod         int64
MonthlyCharges      float64
TotalCharges        float64
Churn                 int64
dtype: object

In [9]:
#Set target, features
feats = list(df)[:-1]
target = list(df)[-1]

#Set X, y values
X = df[feats].values
y = df[target].values

#Train test split
X_train, X_test, y_train, y_test = train_test_split(X,  y, test_size = 0.2, random_state = 42)

for i in X_train, X_test, y_train, y_test:
    print(i.shape)


(5634, 19)
(1409, 19)
(5634,)
(1409,)


In [10]:
y_train

array([0, 0, 1, ..., 0, 1, 0], dtype=int64)

In [11]:
X_train

array([[0.00000e+00, 0.00000e+00, 0.00000e+00, ..., 1.00000e+00,
        6.48500e+01, 1.33680e+03],
       [0.00000e+00, 0.00000e+00, 0.00000e+00, ..., 2.00000e+00,
        9.72000e+01, 5.12945e+03],
       [1.00000e+00, 0.00000e+00, 1.00000e+00, ..., 0.00000e+00,
        2.34500e+01, 2.34500e+01],
       ...,
       [1.00000e+00, 0.00000e+00, 1.00000e+00, ..., 0.00000e+00,
        2.11500e+01, 3.06050e+02],
       [1.00000e+00, 1.00000e+00, 0.00000e+00, ..., 0.00000e+00,
        9.94500e+01, 1.20015e+03],
       [1.00000e+00, 0.00000e+00, 0.00000e+00, ..., 3.00000e+00,
        1.98000e+01, 4.57300e+02]])

In [12]:
#Scale data
ss = StandardScaler()
X_train = ss.fit_transform(X_train)
X_test = ss.transform(X_test)

X_train[0]

array([-1.02516569e+00, -4.37749204e-01, -9.69578591e-01,  1.53218588e+00,
       -4.65683364e-01,  3.29573443e-01, -9.46300240e-01, -1.18234670e+00,
        3.55775250e-01, -1.00654742e+00,  2.90697062e-01, -9.05426063e-01,
       -1.07437424e+00,  2.35570413e-01,  3.72908354e-01, -1.20000722e+00,
       -2.78517888e-01, -4.73723375e-04, -4.21730250e-01])

In [13]:
#Hyperparameters
inputs = X_train.shape[1]
epochs = 100
batch_size = 10


model = Sequential([
    Flatten(),
    Dense(64, activation='selu', input_shape=(inputs,)),
    Dense(64, activation='relu'),
    Dense(1)
])

model.compile(loss='binary_crossentropy', optimizer='adamax', metrics=['mse', 'mae'])

model.fit(X_train, y_train,
         validation_data = (X_test, y_test),
         epochs=epochs,
         batch_size=batch_size)

Train on 5634 samples, validate on 1409 samples
Epoch 1/100
5634/5634 [==============================] - 2s 267us/sample - loss: nan - mse: nan - mae: nan - val_loss: nan - val_mse: nan - val_mae: nan
Epoch 2/100
5634/5634 [==============================] - 1s 149us/sample - loss: nan - mse: nan - mae: nan - val_loss: nan - val_mse: nan - val_mae: nan
Epoch 3/100
5634/5634 [==============================] - 1s 148us/sample - loss: nan - mse: nan - mae: nan - val_loss: nan - val_mse: nan - val_mae: nan
Epoch 4/100
5634/5634 [==============================] - 1s 144us/sample - loss: nan - mse: nan - mae: nan - val_loss: nan - val_mse: nan - val_mae: nan
Epoch 5/100
5634/5634 [==============================] - 1s 168us/sample - loss: nan - mse: nan - mae: nan - val_loss: nan - val_mse: nan - val_mae: nan
Epoch 6/100
5634/5634 [==============================] - 1s 144us/sample - loss: nan - mse: nan - mae: nan - val_loss: nan - val_mse: nan - val_mae: nan
Epoch 7/100
5634/5634 [===========

In [17]:
#GridSearchCV
seed = 10
np.random.seed(seed)

inputs = X_train.shape[1]

def create_model():
    model = Sequential([
        Flatten(),
        Dense(64, activation='selu', input_shape=(input,)),
        Dense(64, activation='tanh'),
        Dense(64, activation='softmax'),
        Dense(1)
    ])
    
    model.compile(loss='binary_crossentropy', optimizer='nadam', metrics=['accuracy'])
    return model

#Build model
model = KerasClassifier(build_fn=create_model, verbose=0)

param_grid = {'batch_size' : [15, 30, 45, 60, 75, 90, 100],
              'epochs' : [10]}

#Create gridsearch

grid = GridSearchCV(estimator=model, param_grid = param_grid, n_jobs=1)
grid_result = grid.fit(X_train, y_train)

#Report result
print(f'Best:{grid_result.best_score_} using {grid_result.best_params_}')
means = grid_result.cv_results_['mean_test_score']
stds = grid_result.cv_results_['std_test_score']
params = grid_result.cv_results_['params']
for mean, stdev, param in zip(means, stds, params):
    print(f'Mean:{mean}, Stdev:{stdev}, with:{param}')

Best:0.734472119808197 using {'batch_size': 15, 'epochs': 10}
Mean:0.734472119808197, Stdev:0.013571901901362286, with:{'batch_size': 15, 'epochs': 10}
Mean:0.734472119808197, Stdev:0.013571901901362286, with:{'batch_size': 30, 'epochs': 10}
Mean:0.734472119808197, Stdev:0.013571901901362286, with:{'batch_size': 45, 'epochs': 10}
Mean:0.734472119808197, Stdev:0.013571901901362286, with:{'batch_size': 60, 'epochs': 10}
Mean:0.734472119808197, Stdev:0.013571901901362286, with:{'batch_size': 75, 'epochs': 10}
Mean:0.734472119808197, Stdev:0.013571901901362286, with:{'batch_size': 90, 'epochs': 10}
Mean:0.734472119808197, Stdev:0.013571901901362286, with:{'batch_size': 100, 'epochs': 10}


In [31]:
#Gridsearch with different parameter
from tensorflow.keras.optimizers import Adamax

input = X_train.shape[1]

def enhanced_model(learn_rate=0.01):
    model = Sequential([
        Dense(64, activation='relu', input_shape=(input,)),
        Dense(64, activation='sigmoid'),
        Dense(64, activation='softmax'),
        Dense(1)
    ])
    model.compile(optimizer=Adamax(lr=learn_rate), loss = 'binary_crossentropy', metrics=['accuracy'])
    
    return model

#Build model
model = KerasClassifier(build_fn=enhanced_model, verbose=0)
lr = [0.01, 0.1, 0.2, 0.3, 0.4]
param_grid = dict(learn_rate=lr)

#Gridsearch
grid = GridSearchCV(estimator=model, param_grid=param_grid, n_jobs=-1, cv=3)
grid_result=grid.fit(X_train, y_train)

#Report result
print(f'Best {grid_result.best_score_} using {grid_result.best_params_}')
means = grid_result.cv_results_['mean_test_score']
stds = grid_result.cv_results_['std_test_score']
params = grid_result.cv_results_['params']
for mean, stdev, param in zip(means, stds, params):
    print(f'Mean :{mean}, Stdev:{stds}, with:{param}')

Best 0.7344692945480347 using {'learn_rate': 0.01}
Mean :0.7344692945480347, Stdev:[0.00714841 0.00714841 0.00714841 0.00714841 0.00714841], with:{'learn_rate': 0.01}
Mean :0.7344692945480347, Stdev:[0.00714841 0.00714841 0.00714841 0.00714841 0.00714841], with:{'learn_rate': 0.1}
Mean :0.7344692945480347, Stdev:[0.00714841 0.00714841 0.00714841 0.00714841 0.00714841], with:{'learn_rate': 0.2}
Mean :0.7344692945480347, Stdev:[0.00714841 0.00714841 0.00714841 0.00714841 0.00714841], with:{'learn_rate': 0.3}
Mean :0.7344692945480347, Stdev:[0.00714841 0.00714841 0.00714841 0.00714841 0.00714841], with:{'learn_rate': 0.4}


## Stretch Goals:

- Try to implement Random Search Hyperparameter Tuning on this dataset
- Try to implement Bayesian Optimiation tuning on this dataset using hyperas or hyperopt (if you're brave)
- Practice hyperparameter tuning other datasets that we have looked at. How high can you get MNIST? Above 99%?
- Study for the Sprint Challenge
 - Can you implement both perceptron and MLP models from scratch with forward and backpropagation?
 - Can you implement both perceptron and MLP models in keras and tune their hyperparameters with cross validation?